## Part 3 Loading Data & EDA

Import relevant packages for the following parts

In [15]:
import numpy as np
import pandas as pd
import math
import re
import sys

from gensim.models.word2vec import Word2Vec

import matplotlib.pyplot as plt
%matplotlib inline
import preprocess

plt.style.use('ggplot')
#from preprocessing import get_processed_data, load_data
import csv
import stanza

from tqdm import tqdm

import time

### Import data cleaned by using the selfwritten preprocessing module

In [4]:
event_df = pd.read_csv('moria_no_duplicates.csv', index_col=0)
# given event date, split the dataset to pre and post event dfs

event_date = '2020-09-09'

#moria_df[['Date','Time']] = moria_df['Date Short'].astype(str).str.split(' ', 1, expand=True)
# create pre and post event partition
pre_event = event_df[event_df['Date Short'] < event_date]
post_event = event_df[event_df['Date Short'] >= event_date]

print('total tweets: ', event_df.shape[0])
print('Pre event tweets: ',pre_event.shape[0])
print('Post event tweets: ',post_event.shape[0])


total tweets:  18203
Pre event tweets:  3311
Post event tweets:  14892


In [5]:
event_df.head()

Date                                                URL  \
0  04-Sep-2020 11:52PM  https://twitter.com/SophieTBHonest/statuses/13...   
1  04-Sep-2020 11:44PM  https://twitter.com/WarWorl/statuses/130199956...   
2  04-Sep-2020 11:36PM  https://twitter.com/brownrecluse101/statuses/1...   
3  04-Sep-2020 10:59PM  https://twitter.com/sbaslan22/statuses/1301988...   
4  04-Sep-2020 10:47PM  https://twitter.com/SophieTBHonest/statuses/13...   

                                           Tweet Raw        Influencer  \
0  @sztiv5 @Juliivan_ Yes, why? Why it wasn’t goo...   @sophietbhonest   
1  @GoTurkey ISIS refuge. Wouldn't go to Turkey i...          @warworl   
2  Greece must improve refugee overcrowding, UN w...  @brownrecluse101   
3  @ThisIsOzcan @Nervana_1 @EGozuguzelli 1/3 Law?...        @sbaslan22   
4  @Juliivan_ @sztiv5 Anyway, how did the asylum ...   @sophietbhonest   

          Country Language  Reach  Engagement    AVE Sentiment  ...      Time  \
0       Australia  English     45         NaN   0.42   Neutral  ...  11:52 PM   
1  United Kingdom  English      5         NaN   0.05   Neutral  ...  11:44 PM   
2         Unknown  English   1348         NaN  12.47   Neutral  ...  11:36 PM   
3          Turkey  English    315         NaN   2.91   Neutral  ...  10:59 PM   
4       Australia  English     45         NaN   0.42   Neutral  ...  10:47 PM   

     State        City Hashtags                                      Mentions  \
0      NaN         NaN       []                       ['sztiv5', 'Juliivan_']   
1  England  Ravenstone       []                                  ['GoTurkey']   
2      NaN         NaN       []                                            []   
3   Edirne   Uzunköprü       []  ['ThisIsOzcan', 'Nervana_1', 'EGozuguzelli']   
4      NaN         NaN       []                       ['Juliivan_', 'sztiv5']   

                Linked Content Retweet Quote Tweet  \
0                           []   False       False   
1                           []   False       False   
2  ['https://t.co/UDM4GDMcmo']   False       False   
3                           []   False       False   
4                           []   False       False   

                                         Tweet Clean  \
0  yes, why? why it wasnt good to apply for asylu...   
1  isis refuge. wouldn't go to turkey if i was pa...   
2  greece must improve refugee overcrowding, un w...   
3  /3 law? let the idle stuff. all rights of the ...   
4  anyway, how did the asylum seekers ended up at...   

                                  Tweet Clean Tokens  
0  ['wasnt', 'good', 'apply', 'asylum', 'greece',...  
1  ['refuge', 'would', 'turkey', 'paid', 'likely'...  
2  ['greece', 'must', 'improve', 'refugee', 'over...  
3  ['idle', 'stuff', 'right', 'turkish', 'minorit...  
4  ['anyway', 'asylum', 'seeker', 'ended', 'borde...  

[5 rows x 32 columns]

In [6]:
list(event_df["Tweet Raw"])[:100]

['@sztiv5 @Juliivan_ Yes, why? Why it wasn’t good to apply for asylum in Greece, MAC, SER or CRO or BUL, together 3 EU members before HU? They must get help in the first safe country as asylum seeker,not in the 5th. No law says you can pick and choose and get it.',
 "@GoTurkey ISIS refuge. Wouldn't go to Turkey if I was paid. You're likely to get your head lopped off if you stray off the beaten track. Go to Christian Greece and be safe while enjoying a similar climate to islamic turkey",
 'Greece must improve refugee overcrowding, UN warns https://t.co/UDM4GDMcmo',
 '@ThisIsOzcan @Nervana_1 @EGozuguzelli 1/3 Law? Let the idle stuff. All rights of the Turkish minority in Greece were taken away. Where is the law? Refugees are not accepted into the EU. Where is the law? Western states divided the states in the Middle East and Africa for underground resources. Where is the law?',
 '@Juliivan_ @sztiv5 Anyway, how did the asylum seekers ended up at HU borders? They must have had a long journ

In [7]:
stanza.download("en")

2021-02-23 22:11:49 INFO: Downloading default packages for language: en (English)...
2021-02-23 22:15:26 INFO: Finished downloading models and saved to C:\Users\nikodemicek\stanza_resources.


In [8]:
en_nlp = stanza.Pipeline("en")

2021-02-23 22:15:26 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-02-23 22:15:26 INFO: Use device: cpu
2021-02-23 22:15:26 INFO: Loading: tokenize
2021-02-23 22:15:26 INFO: Loading: pos
2021-02-23 22:15:26 INFO: Loading: lemma
2021-02-23 22:15:26 INFO: Loading: depparse
2021-02-23 22:15:27 INFO: Loading: sentiment
2021-02-23 22:15:28 INFO: Loading: ner
2021-02-23 22:15:29 INFO: Done loading processors!


In [9]:
start = time.time()
en_doc = event_df["Tweet Raw"][:100].apply(en_nlp)
end = time.time()
print(f"Preprocessing the data took {end-start} seconds.")

Preprocessing the data took 201.91080117225647 seconds.


In [10]:
en_doc.iloc[1].sentences[0]

[
  {
    "id": 1,
    "text": "@GoTurkey",
    "lemma": "@GoTurkey",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 3,
    "deprel": "compound",
    "misc": "start_char=0|end_char=9",
    "ner": "O"
  },
  {
    "id": 2,
    "text": "ISIS",
    "lemma": "ISIS",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 3,
    "deprel": "compound",
    "misc": "start_char=10|end_char=14",
    "ner": "S-ORG"
  },
  {
    "id": 3,
    "text": "refuge",
    "lemma": "refuge",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 0,
    "deprel": "root",
    "misc": "start_char=15|end_char=21",
    "ner": "O"
  },
  {
    "id": 4,
    "text": ".",
    "lemma": ".",
    "upos": "PUNCT",
    "xpos": ".",
    "head": 3,
    "deprel": "punct",
    "misc": "start_char=21|end_char=22",
    "ner": "O"
  }
]

In [11]:
directory = None ##ADD DIRECTORY HERE
stanza.install_corenlp(directory)

2021-02-23 22:20:31 INFO: Installing CoreNLP package into export CORENLP_HOME=/path/to/stanford-corenlp-4.1.0...


In [24]:
from stanza.server import CoreNLPClient

# get noun phrases with tregex
def noun_phrases(_client, _text, _annotators=None):
    pattern = 'NP'
    matches = _client.tregex(_text,pattern,annotators=_annotators)
    return [sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]


In [34]:
noun_phrase_list = []
anns_list = []
with CoreNLPClient(timeout=30000, memory='16G') as client:
    for tweet in tqdm(event_df["Tweet Raw"][:100]):
        noun_phrase = noun_phrases(client,str(tweet),_annotators="tokenize,ssplit,pos,lemma,parse")
        noun_phrase_list.append(noun_phrase)
        #noun_phrases(client,event_df["Tweet Raw"][:100],_annotators="tokenize,ssplit,pos,lemma,parse")

2021-02-23 23:18:21 INFO: Writing properties to tmp file: corenlp_server-dfd06ada3d5049c4.props
2021-02-23 23:18:21 INFO: Starting server with command: java -Xmx16G -cp export CORENLP_HOME=/path/to/stanford-corenlp-4.1.0\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-dfd06ada3d5049c4.props -preload -outputFormat serialized
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.32s/it]


In [36]:
noun_phrase_list


[['@sztiv5 @Juliivan_ Yes, why?',
  '@sztiv5',
  '@Juliivan_ Yes, why',
  '@Juliivan_',
  'it',
  'asylum in Greece, MAC, SER or CRO or BUL',
  'asylum in Greece, MAC, SER or CRO',
  'asylum',
  'Greece, MAC, SER or CRO',
  'BUL',
  '3 EU members',
  'HU',
  'They',
  'help',
  'the first safe country as asylum seeker',
  'the first safe country',
  'asylum seeker',
  'the 5th',
  'No law',
  'you',
  'it'],
 ['@GoTurkey ISIS',
  'refuge',
  'Turkey',
  'I',
  'You',
  'your head',
  'you',
  'the beaten track',
  'Christian Greece',
  'a similar climate',
  'islamic turkey'],
 ['Greece', 'refugee overcrowding', 'UN', 'https://t.co/UDM4GDMcmo'],
 ['@ThisIsOzcan @Nervana_1 @EGozuguzelli',
  '1/3',
  'Law',
  'the idle stuff',
  'All rights of the Turkish minority in Greece',
  'All rights',
  'the Turkish minority in Greece',
  'the Turkish minority',
  'Greece',
  'the law',
  'Refugees',
  'the EU',
  'the law',
  'Western states',
  'the states',
  'the Middle East and Africa',
  'th